In [1]:
#Importamos las librerias necesarias para crear la red neuronal
import pandas
import numpy as np
from sklearn import model_selection, preprocessing
from tensorflow import keras
import tensorflow as tf
#Leemos el csv
evaluation = pandas.read_csv('turkiye-student-evaluation_R_Specific.csv', header=0)

In [2]:
#Mostramos las 10 primeras lineas para ver que se ha leido bien
evaluation.head(10)

,instr,class,nb.repeat,attendance,difficulty,Q1,Q2,Q3,Q4,Q5,...,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28
1,1,2,1,0,4,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2,1,2,1,1,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
3,1,2,1,2,4,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
4,1,2,1,1,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
5,1,2,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6,1,2,1,3,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
7,1,2,1,1,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
8,1,2,1,1,3,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
9,1,2,1,1,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
10,1,2,1,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [3]:
#Seleccionamos los atributos 
atributos = evaluation.loc[:, 'class':'Q27']
atributos = atributos.to_numpy()
scaler = preprocessing.MinMaxScaler()
atributos = scaler.fit_transform(atributos)
print(atributos)

[[0.08333333 0.         0.         ... 0.5        0.5        0.5       ]
 [0.08333333 0.         0.25       ... 0.5        0.5        0.5       ]
 [0.08333333 0.         0.5        ... 1.         1.         1.        ]
 ...
 [1.         0.         0.         ... 1.         1.         1.        ]
 [1.         0.         0.25       ... 0.         0.         0.        ]
 [1.         0.         0.25       ... 0.         0.         0.        ]]


In [4]:
objetivo = evaluation['Q28']
objetivo = evaluation['Q28'].to_numpy(dtype=np.float32)
scaler = preprocessing.MinMaxScaler()
objetivo = scaler.fit_transform(objetivo.reshape(-1, 1))
print(objetivo)

[[0.5]
 [0.5]
 [1. ]
 ...
 [1. ]
 [0. ]
 [0. ]]


In [5]:
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = model_selection.train_test_split(
    atributos, objetivo, test_size=.15)

In [6]:
normalizador = keras.layers.Normalization()
normalizador.adapt(atributos_entrenamiento)
red_evaluation = keras.Sequential()
red_evaluation.add(keras.layers.Input(shape=(31,)))
red_evaluation.add(normalizador)
red_evaluation.add(keras.layers.Dense(70, activation='relu'))
red_evaluation.add(keras.layers.Dense(70, activation='sigmoid'))
red_evaluation.add(keras.layers.Dense(1, activation='sigmoid'))
red_evaluation.add(keras.layers.Lambda(lambda x: x * 5))


In [7]:
red_evaluation.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 31)               63        
 n)                                                              
                                                                 
 dense (Dense)               (None, 70)                2240      
                                                                 
 dense_1 (Dense)             (None, 70)                4970      
                                                                 
 dense_2 (Dense)             (None, 1)                 71        
                                                                 
 lambda (Lambda)             (None, 1)                 0         
                                                                 
Total params: 7,344
Trainable params: 7,281
Non-trainable params: 63
_____________________________________________________

In [8]:

red_evaluation.compile(optimizer='SGD', loss='mean_squared_error', metrics=['accuracy', 'mae'])
red_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento,
                batch_size=256, epochs=10)

Epoch 1/10
20/20 [==============================] - 1s 5ms/step - loss: 0.4648 - accuracy: 0.2695 - mae: 0.4289
Epoch 2/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0591 - accuracy: 0.3273 - mae: 0.2059
Epoch 3/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0488 - accuracy: 0.3277 - mae: 0.1854
Epoch 4/10
20/20 [==============================] - 0s 5ms/step - loss: 0.0415 - accuracy: 0.3277 - mae: 0.1691
Epoch 5/10
20/20 [==============================] - 0s 5ms/step - loss: 0.0362 - accuracy: 0.3283 - mae: 0.1560
Epoch 6/10
20/20 [==============================] - 0s 5ms/step - loss: 0.0324 - accuracy: 0.3279 - mae: 0.1453
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0296 - accuracy: 0.3273 - mae: 0.1365
Epoch 8/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0275 - accuracy: 0.3275 - mae: 0.1293
Epoch 9/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0259 - accuracy: 0.3277 - mae:

In [9]:
#Definimos cuanta perdida queremos tener y ejecutamos el siguiente bucle para entrenar la red hasta conseguir una perdida menor a la definida
perdObj = 1.0
x=red_evaluation.evaluate(atributos_prueba, objetivo_prueba)
while x[0]>perdObj:
    red_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento,
                batch_size=256, epochs=10)
    x=red_evaluation.evaluate(atributos_prueba, objetivo_prueba)
    
print(x[0])

28/28 [==============================] - 0s 3ms/step - loss: 0.0256 - accuracy: 0.3150 - mae: 0.1188
0.025632139295339584


In [10]:
#Ahora que ya hemos conseguido una perdida menor a la que queriamos, evaluamos para volver a comprobar que la hemos conseguido
red_evaluation.evaluate(atributos_prueba, objetivo_prueba)

28/28 [==============================] - 0s 4ms/step - loss: 0.0256 - accuracy: 0.3150 - mae: 0.1188


[0.025632139295339584, 0.31500571966171265, 0.11876217275857925]

In [11]:
#Ahora mismo comprobaremos manualmente si las predicciones de la red se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado

X=atributos[5809]
predicciones = red_evaluation.predict(X)
print('Esperado 4:',predicciones*5)
X=atributos[5810]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones*5)
X=atributos[5811]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones*5)
X=atributos[5812]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones*5)
X=atributos[5813]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones*5)
X=atributos[5814]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones*5)
X=atributos[5815]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones*5)
X=atributos[5816]
predicciones = red_evaluation.predict(X)
print('Esperado 5:',predicciones*5)
X=atributos[5817]
predicciones = red_evaluation.predict(X)
print('Esperado 5:',predicciones*5)
X=atributos[5818]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones*5)
X=atributos[5819]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones*5)

1/1 [==============================] - 0s 228ms/step
Esperado 4: [[4.253561]]
1/1 [==============================] - 0s 50ms/step
Esperado 3: [[2.8934164]]
1/1 [==============================] - 0s 49ms/step
Esperado 3: [[2.4374611]]
1/1 [==============================] - 0s 55ms/step
Esperado 1: [[0.9049922]]
1/1 [==============================] - 0s 51ms/step
Esperado 3: [[2.5657332]]
1/1 [==============================] - 0s 50ms/step
Esperado 1: [[0.9890482]]
1/1 [==============================] - 0s 47ms/step
Esperado 1: [[0.9049922]]
1/1 [==============================] - 0s 55ms/step
Esperado 5: [[4.687421]]
1/1 [==============================] - 0s 50ms/step
Esperado 5: [[5.1095815]]
1/1 [==============================] - 0s 51ms/step
Esperado 1: [[0.8890904]]
1/1 [==============================] - 0s 47ms/step
Esperado 1: [[0.8890904]]


In [12]:
#Ahora trabajaremos con el otro modelo que será un random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
forest_evaluation = RandomForestRegressor()
forest_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento)
evaluaciones = forest_evaluation.predict(atributos_prueba)
mse = mean_squared_error(objetivo_prueba, evaluaciones)
print("Error cuadrático medio (MSE) del modelo:", mse)

C:\Users\anton\AppData\Local\Temp\ipykernel_21260\849215698.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento)


Error cuadrático medio (MSE) del modelo: 0.014143653716483693


In [13]:
#Ahora mismo comprobaremos manualmente si las predicciones del random forest se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado
atributos2 = atributos[:, np.newaxis]
X=atributos2[5809]
predicciones = forest_evaluation.predict(X)
print('Esperado 4:',predicciones*5)
X=atributos2[5810]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones*5)
X=atributos2[5811]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones*5)
X=atributos2[5812]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones*5)
X=atributos2[5813]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones*5)
X=atributos2[5814]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones*5)
X=atributos2[5815]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones*5)
X=atributos2[5816]
predicciones = forest_evaluation.predict(X)
print('Esperado 5:',predicciones*5)
X=atributos2[5817]
predicciones = forest_evaluation.predict(X)
print('Esperado 5:',predicciones*5)
X=atributos2[5818]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones*5)
X=atributos2[5819]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones*5)

Esperado 4: [3.75]
Esperado 3: [2.5]
Esperado 3: [2.5]
Esperado 1: [0.]
Esperado 3: [2.5]
Esperado 1: [0.1]
Esperado 1: [0.]
Esperado 5: [4.915625]
Esperado 5: [5.]
Esperado 1: [0.]
Esperado 1: [0.]


In [1]:
#Importamos las librerias necesarias para crear la red neuronal
import pandas
import numpy as np
from sklearn import model_selection, preprocessing
from tensorflow import keras
import tensorflow as tf

#Leemos el csv
#bikes = pandas.read_csv('hour.csv', header=0)
bikes = pandas.read_csv('hour2.csv',header = 0)


In [2]:
#Mostramos las 10 primeras lineas para ver que se ha leido bién
bikes.head(10)

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
5,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1
6,1,0,1,6,0,6,0,1,0.22,0.2727,0.80,0.0000,2,0,2
7,1,0,1,7,0,6,0,1,0.20,0.2576,0.86,0.0000,1,2,3
8,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0.0000,1,7,8
9,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0.0000,8,6,14


In [3]:
#Seleccionamos los atributos y los nomalizamos
attr = bikes.loc[:, 'season':'registered']
attr = attr.to_numpy()
scaler = preprocessing.MinMaxScaler()
attr = scaler.fit_transform(attr)
print(attr)

[[0.         0.         0.         ... 0.         0.00817439 0.01467269]
 [0.         0.         0.         ... 0.         0.02179837 0.03611738]
 [0.         0.         0.         ... 0.         0.01362398 0.03047404]
 ...
 [0.         1.         1.         ... 0.19301751 0.01907357 0.09367946]
 [0.         1.         1.         ... 0.15786999 0.03542234 0.05417607]
 [0.         1.         1.         ... 0.15786999 0.03269755 0.04176072]]


In [4]:
#Seleccionamos los objetivos
obj = bikes['cnt']
obj = bikes['cnt'].to_numpy(dtype=np.float32)
#scaler = preprocessing.MinMaxScaler()
#objetivo = scaler.fit_transform(objetivo.reshape(-1, 1))
print(obj)

[16. 40. 32. ... 90. 61. 49.]


In [5]:
#Cogemos un 85% de los datos para entrenar y un 15% de los datos para evaluar el modelo
(attr_entrenamiento, attr_prueba,
 obj_entrenamiento, obj_prueba) = model_selection.train_test_split(
    attr, obj, test_size=.10)

In [6]:
#Creamos la red neuronal 
normalizador = keras.layers.Normalization()
normalizador.adapt(attr_entrenamiento)
red_bikes = keras.Sequential()
red_bikes.add(keras.layers.Input(shape=(14,))) #añadimos la capa de entrada con 14 neuronas
red_bikes.add(normalizador)#Aplicamos un normalizador
red_bikes.add(keras.layers.Dense(100, activation='sigmoid'))#Añadimos una capa intermedia de 100 neuronas con función de activación sigmoide
red_bikes.add(keras.layers.Dense(100, activation='sigmoid'))#Añadimos una capa intermedia de 100 neuronas con función de activación sigmoide
red_bikes.add(keras.layers.Dense(1, activation='linear'))#Necesitamos una capa de salida linear ya que los valores objetivos son lineares
#red_bikes.add(keras.layers.Lambda(lambda x: tf.cast(tf.round(x), dtype=tf.int32)))
#red_bikes.add(keras.layers.Lambda(lambda x: round(x)))


In [7]:
#Vemos un resumen de la red
red_bikes.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 14)               29        
 n)                                                              
                                                                 
 dense (Dense)               (None, 100)               1500      
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 11,730
Trainable params: 11,701
Non-trainable params: 29
_________________________________________________________________


In [8]:
#Compilamos y entrenamos la red neuronal, buscamos minimizar la función de perdida
red_bikes.compile(optimizer='SGD', loss='mean_squared_error', metrics=['accuracy', 'mae'])
red_bikes.fit(attr_entrenamiento, obj_entrenamiento,
                batch_size=256, epochs=10)

Epoch 1/10
62/62 [==============================] - 2s 5ms/step - loss: 12021.9287 - accuracy: 0.0092 - mae: 73.1732
Epoch 2/10
62/62 [==============================] - 0s 4ms/step - loss: 2112.9321 - accuracy: 0.0093 - mae: 33.8189
Epoch 3/10
62/62 [==============================] - 0s 5ms/step - loss: 967.8004 - accuracy: 0.0093 - mae: 21.8101
Epoch 4/10
62/62 [==============================] - 0s 5ms/step - loss: 685.3350 - accuracy: 0.0093 - mae: 18.3104
Epoch 5/10
62/62 [==============================] - 0s 5ms/step - loss: 398.0434 - accuracy: 0.0093 - mae: 14.9239
Epoch 6/10
62/62 [==============================] - 0s 5ms/step - loss: 393.9950 - accuracy: 0.0093 - mae: 14.4092
Epoch 7/10
62/62 [==============================] - 0s 4ms/step - loss: 145.8375 - accuracy: 0.0093 - mae: 8.7118
Epoch 8/10
62/62 [==============================] - 0s 5ms/step - loss: 220.8146 - accuracy: 0.0093 - mae: 10.9439
Epoch 9/10
62/62 [==============================] - 0s 4ms/step - loss: 112.58

In [26]:
#Definimos cuanta perdida queremos tener y ejecutamos el siguiente bucle para entrenar la red hasta conseguir una perdida menor a la definida
perdidaObj = 1.0
x=red_bikes.evaluate(attr_prueba, obj_prueba)
while x[0]>perdidaObj:
    red_bikes.fit(attr_entrenamiento, obj_entrenamiento,
                batch_size=256, epochs=10)
    x=red_bikes.evaluate(attr_prueba, obj_prueba)
    
print(x[0])

55/55 [==============================] - 0s 3ms/step - loss: 90.7335 - accuracy: 0.0052 - mae: 5.2598
Epoch 1/10
62/62 [==============================] - 0s 5ms/step - loss: 112.5309 - accuracy: 0.0095 - mae: 7.8190
Epoch 2/10
62/62 [==============================] - 0s 5ms/step - loss: 75.3027 - accuracy: 0.0095 - mae: 6.1970
Epoch 3/10
62/62 [==============================] - 0s 4ms/step - loss: 51.7147 - accuracy: 0.0095 - mae: 5.3568
Epoch 4/10
62/62 [==============================] - 0s 4ms/step - loss: 56.8056 - accuracy: 0.0095 - mae: 5.3858
Epoch 5/10
62/62 [==============================] - 0s 5ms/step - loss: 56.0395 - accuracy: 0.0095 - mae: 5.1108
Epoch 6/10
62/62 [==============================] - 0s 5ms/step - loss: 45.7205 - accuracy: 0.0095 - mae: 5.2062
Epoch 7/10
62/62 [==============================] - 0s 5ms/step - loss: 34.9572 - accuracy: 0.0095 - mae: 4.5593
Epoch 8/10
62/62 [==============================] - 0s 5ms/step - loss: 31.1880 - accuracy: 0.0095 - mae: 

62/62 [==============================] - 0s 5ms/step - loss: 2.2360 - accuracy: 0.0095 - mae: 1.1359
Epoch 8/10
62/62 [==============================] - 0s 5ms/step - loss: 8.3165 - accuracy: 0.0095 - mae: 2.3805
Epoch 9/10
62/62 [==============================] - 0s 5ms/step - loss: 16.5208 - accuracy: 0.0095 - mae: 3.0768
Epoch 10/10
55/55 [==============================] - 0s 3ms/step - loss: 3.8090 - accuracy: 0.0052 - mae: 1.4934
Epoch 1/10
62/62 [==============================] - 0s 5ms/step - loss: 1.4122 - accuracy: 0.0095 - mae: 0.8418
Epoch 2/10
62/62 [==============================] - 0s 5ms/step - loss: 10.8872 - accuracy: 0.0095 - mae: 2.4230
Epoch 3/10
62/62 [==============================] - 0s 5ms/step - loss: 9.7188 - accuracy: 0.0095 - mae: 2.4767
Epoch 4/10
62/62 [==============================] - 0s 5ms/step - loss: 1.2226 - accuracy: 0.0095 - mae: 0.7982
Epoch 5/10
62/62 [==============================] - 0s 4ms/step - loss: 2.9209 - accuracy: 0.0095 - mae: 1.3266


62/62 [==============================] - 0s 4ms/step - loss: 5.4853 - accuracy: 0.0095 - mae: 1.7747
Epoch 4/10
62/62 [==============================] - 0s 5ms/step - loss: 12.2843 - accuracy: 0.0095 - mae: 2.7336
Epoch 5/10
62/62 [==============================] - 0s 5ms/step - loss: 0.7191 - accuracy: 0.0095 - mae: 0.6258
Epoch 6/10
62/62 [==============================] - 0s 5ms/step - loss: 1.1090 - accuracy: 0.0095 - mae: 0.7720
Epoch 7/10
62/62 [==============================] - 0s 5ms/step - loss: 2.9201 - accuracy: 0.0095 - mae: 1.3656
Epoch 8/10
62/62 [==============================] - 0s 4ms/step - loss: 7.2539 - accuracy: 0.0095 - mae: 2.1287
Epoch 9/10
62/62 [==============================] - 0s 5ms/step - loss: 5.1332 - accuracy: 0.0095 - mae: 1.6315
Epoch 10/10
55/55 [==============================] - 0s 3ms/step - loss: 1.0302 - accuracy: 0.0052 - mae: 0.8152
Epoch 1/10
62/62 [==============================] - 0s 5ms/step - loss: 0.8952 - accuracy: 0.0095 - mae: 0.7237
E

55/55 [==============================] - 0s 3ms/step - loss: 10.1939 - accuracy: 0.0052 - mae: 2.3820
Epoch 1/10
62/62 [==============================] - 0s 5ms/step - loss: 4.3022 - accuracy: 0.0095 - mae: 1.6503
Epoch 2/10
62/62 [==============================] - 0s 5ms/step - loss: 12.8244 - accuracy: 0.0095 - mae: 2.9578
Epoch 3/10
62/62 [==============================] - 0s 4ms/step - loss: 1.9915 - accuracy: 0.0095 - mae: 1.0963
Epoch 4/10
62/62 [==============================] - 0s 5ms/step - loss: 0.8260 - accuracy: 0.0095 - mae: 0.6252
Epoch 5/10
62/62 [==============================] - 0s 5ms/step - loss: 2.8837 - accuracy: 0.0095 - mae: 1.3680
Epoch 6/10
62/62 [==============================] - 0s 4ms/step - loss: 4.0398 - accuracy: 0.0095 - mae: 1.5607
Epoch 7/10
62/62 [==============================] - 0s 4ms/step - loss: 3.1580 - accuracy: 0.0095 - mae: 1.3636
Epoch 8/10
62/62 [==============================] - 0s 5ms/step - loss: 3.0430 - accuracy: 0.0095 - mae: 1.4530
E

In [210]:
#Ahora que ya hemos conseguido una perdida menor a la que queriamos, evaluamos para volver a comprobar que la hemos conseguido
red_bikes.evaluate(attr_prueba, obj_prueba)

55/55 [==============================] - 0s 1ms/step - loss: 0.8929 - accuracy: 0.0081 - mae: 0.6922


[0.8928684592247009, 0.00805523619055748, 0.6922006011009216]

In [20]:
#Ahora mismo comprobaremos manualmente si las predicciones de la red se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado

X=attr[17368]
predicciones = red_bikes.predict(X)
print('Esperado 203:',predicciones)
X=attr[17369]
predicciones = red_bikes.predict(X)
print('Esperado 247:',predicciones)
X=attr[17370]
predicciones = red_bikes.predict(X)
print('Esperado 315:',predicciones)
X=attr[17371]
predicciones = red_bikes.predict(X)
print('Esperado 214:',predicciones)
X=attr[17372]
predicciones = red_bikes.predict(X)
print('Esperado 164:',predicciones)
X=attr[17373]
predicciones = red_bikes.predict(X)
print('Esperado 122:',predicciones)
X=attr[17374]
predicciones = red_bikes.predict(X)
print('Esperado 119:',predicciones)
X=attr[17375]
predicciones = red_bikes.predict(X)
print('Esperado 89:',predicciones)
X=attr[17376]
predicciones = red_bikes.predict(X)
print('Esperado 90:',predicciones)
X=attr[17377]
predicciones = red_bikes.predict(X)
print('Esperado 61:',predicciones)
X=attr[17378]
predicciones = red_bikes.predict(X)
print('Esperado 49:',predicciones)

1/1 [==============================] - 0s 28ms/step
Esperado 203: [[203.3557]]
1/1 [==============================] - 0s 22ms/step
Esperado 247: [[246.93541]]
1/1 [==============================] - 0s 26ms/step
Esperado 315: [[315.41962]]
1/1 [==============================] - 0s 25ms/step
Esperado 214: [[215.09206]]
1/1 [==============================] - 0s 26ms/step
Esperado 164: [[164.07002]]
1/1 [==============================] - 0s 22ms/step
Esperado 122: [[121.828186]]
1/1 [==============================] - 0s 37ms/step
Esperado 119: [[119.20893]]
1/1 [==============================] - 0s 30ms/step
Esperado 89: [[89.70702]]
1/1 [==============================] - 0s 27ms/step
Esperado 90: [[90.72455]]
1/1 [==============================] - 0s 23ms/step
Esperado 61: [[62.387413]]
1/1 [==============================] - 0s 25ms/step
Esperado 49: [[48.95525]]


In [25]:
#Ahora trabajaremos con el otro modelo que será un random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
forest_bikes = RandomForestRegressor()
forest_bikes.fit(attr_entrenamiento, obj_entrenamiento)
evaluaciones = forest_bikes.predict(attr_prueba)
mse = mean_squared_error(obj_prueba, evaluaciones)
print("Error cuadrático medio (MSE) del modelo:", mse)

Error cuadrático medio (MSE) del modelo: 3.8617047756041427


In [33]:
#Ahora mismo comprobaremos manualmente si las predicciones del random forest se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado
attr2 = attr[:, np.newaxis]
X=attr2[17368]
predicciones = forest_bikes.predict(X)
print('Esperado 203:',predicciones)
X=attr2[17369]
predicciones = forest_bikes.predict(X)
print('Esperado 247:',predicciones)
X=attr2[17370]
predicciones = forest_bikes.predict(X)
print('Esperado 315:',predicciones)
X=attr2[17371]
predicciones = forest_bikes.predict(X)
print('Esperado 214:',predicciones)
X=attr2[17372]
predicciones = forest_bikes.predict(X)
print('Esperado 164:',predicciones)
X=attr2[17373]
predicciones = forest_bikes.predict(X)
print('Esperado 122:',predicciones)
X=attr2[17374]
predicciones = forest_bikes.predict(X)
print('Esperado 119:',predicciones)
X=attr2[17375]
predicciones = forest_bikes.predict(X)
print('Esperado 89:',predicciones)
X=attr2[17376]
predicciones = forest_bikes.predict(X)
print('Esperado 90:',predicciones)
X=attr2[17377]
predicciones = forest_bikes.predict(X)
print('Esperado 61:',predicciones)
X=attr2[17378]
predicciones = forest_bikes.predict(X)
print('Esperado 49:',predicciones)

Esperado 203: [202.88]
Esperado 247: [246.91]
Esperado 315: [314.35]
Esperado 214: [214.84]
Esperado 164: [163.97]
Esperado 122: [122.11]
Esperado 119: [118.92]
Esperado 89: [89.09]
Esperado 90: [90.2]
Esperado 61: [61.13]
Esperado 49: [49.11]


In [ ]:
#Metodo LIME
import random
from sklearn.linear_model import Ridge



"""
N es el n´umero de permutaciones a realizar
f es el modelo a explicar
X′ ← {} muestras perturbadas
R ← {} representaciones
W ← {} las distancias entre la muestra x y sus perturbaciones
for 1
to
N do
Selecciona k atributos aleatoriamente
x′ ← una perturbaci´on de x donde se perturban los k atributos anteriores.
w ← la distancia entre x y x′
r ← la representaci´on de x′ respecto a x
X′ ← X′ ∪ x′
R ← R ∪ r
W ← R ∪ w
end for
Y ′ ← f(X′) las predicciones de las perturbaciones
G ← modelo ridge entrenado con R para predecir Y ′ y ponderando cada
muestra con W
return los par´ametros de G
"""

def explain_model(f, x, N,M):
    #f es el modelo a explicar
    #N es el número de permutaciones a realizar
    Xi = [] #muestras perturbadas
    R = [] #representaciones
    W = [] #Las distancias entre la muestra x y sus pertubaciones
    for i in range(N):
        k = random.randint(1, len(x)) #Escojo un número aletario que representa el número de los atributos a seleccionar
        perturbed_x = x.copy() #copio la muestra original
        for i in range(k): 
            perturbed_attr = random.choice(list(x.keys())) #Voy escojiento los atributos que perturbare de forma aleatoria
            mx = abs(max([a[perturbed_attr] for a in M])) #Para el atributo aletorio seleccionado calculo el valor máximo
            mn = abs(min([a[perturbed_attr] for a in M])) #Para el atributo aletorio seleccionado calculo el valor mínimo
            perturbed_x[perturbed_attr] = random.uniform(mn, mx) # pertubo el atributo con un valor aleatorio ente mn y mx
        w = abs(sum([x[attr] - perturbed_x[attr] for attr in x.keys()])) #Calculo la distancia entre el original y el perturbado
        r = {attr: 0  if x_perturbed[attr] = x[attr] else 1 for attr in x.keys()} #miramos la representacion del pertubado respecto al original utilizando un operador ternario como solemos hacer en java
        Xi.append(x_perturbed) #Acumulo en la lista los perturbados
        R.append(r)#Acumulamos en la lista de representaciones
        W.append(w)#Añadimos las distancias a la lista
    Y_perturbed = f(Xi)#aplicamos el modelo f
    G = Ridge().fit(R, Y_perturbed, sample_weight=W) #Utilizamos Ridge 
    return G.get_params() #Devolvemos los parametros


In [ ]:
#METODO IDENTIDAD
import pandas
import numpy as np
from scipy.spatial.distance import cdist
from scipy.stats import pearsonr
from scipy.special import lambertw
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Creamos dos instancias de datos y sus respectivas explicaciones LIME
xa = np.array([0.5, 0.7, 0.1, 0.3])
xb = np.array([0.3, 0.9, 0.2, 0.1])
ea = np.array([0.2, 0.3, 0.1, 0.4])
eb = np.array([0.3, 0.4, 0.5, 0.1])

# Calculamos la distancia entre las instancias de datos
data_distance = cdist(xa.reshape(1,-1), xb.reshape(1,-1))

# Calculamos la distancia entre las explicaciones LIME
exp_distance = cdist(ea.reshape(1,-1), eb.reshape(1,-1))

# Verificamos si la métrica Identidad se cumple
if data_distance == 0 and exp_distance != 0:
    print("La métrica Identidad no se cumple para este par de instancias de datos y sus explicaciones LIME correspondientes.")
else:
    print("La métrica Identidad se cumple para este par de instancias de datos y sus explicaciones LIME correspondientes.")


In [ ]:
#METODO SEPARABILIDAD
# Creamos dos instancias de datos y sus respectivas explicaciones LIME
xa = np.array([0.5, 0.7, 0.1, 0.3])
xb = np.array([0.3, 0.9, 0.2, 0.1])
ea = np.array([0.2, 0.3, 0.1, 0.4])
eb = np.array([0.3, 0.4, 0.5, 0.1])

# Calculamos la distancia entre las instancias de datos
data_distance = cdist(xa.reshape(1,-1), xb.reshape(1,-1))

# Calculamos la distancia entre las explicaciones LIME
exp_distance = cdist(ea.reshape(1,-1), eb.reshape(1,-1))

# Verificamos si la métrica Separabilidad se cumple
if data_distance != 0 and exp_distance <= 0:
    print("La métrica Separabilidad no se cumple para este par de instancias de datos y sus explicaciones LIME correspondientes.")
else:
    print("La métrica Separabilidad se cumple para este par de instancias de datos y sus explicaciones LIME correspondientes.")


In [ ]:
#METODO ESTABILIDAD
# Creamos las instancias de datos y sus respectivas explicaciones LIME
x = np.array([[0.5, 0.7, 0.1, 0.3], [0.3, 0.9, 0.2, 0.1], [0.1, 0.2, 0.8, 0.6]])
ex = np.array([[0.2, 0.3, 0.1, 0.4], [0.3, 0.4, 0.5, 0.1], [0.1, 0.3, 0.6, 0.2]])

# Creamos las distancias entre las instancias de datos y sus respectivas explicaciones LIME
data_distances = cdist(x, x[0].reshape(1,-1))
exp_distances = cdist(ex, ex[0].reshape(1,-1))

# Calculamos el coeficiente de correlación de Pearson entre las distancias
correlation, _ = pearsonr(data_distances.flatten(), exp_distances.flatten())

# Verificamos si la métrica Estabilidad se cumple
if correlation > 0:
    print("La métrica Estabilidad se cumple para este conjunto de instancias de datos y sus explicaciones LIME correspondientes.")
else:
    print("La métrica Estabilidad no se cumple para este conjunto de instancias de datos y sus explicaciones LIME correspondientes.")


In [ ]:
#METODO COHERENCIA
# Creamos las instancias de datos y sus respectivas explicaciones LIME
x = np.array([[0.5, 0.7, 0.1, 0.3], [0.3, 0.9, 0.2, 0.1], [0.1, 0.2, 0.8, 0.6]])
ex = np.array([[0.2, 0.3, 0.1, 0.4], [0.3, 0.4, 0.5, 0.1], [0.1, 0.3, 0.6, 0.2]])

# Calculamos los pesos de las instancias de datos y sus respectivas explicaciones LIME
data_weights = np.exp(-np.linalg.norm(x - x.mean(axis=0), axis=1))
exp_weights = np.exp(-np.linalg.norm(ex - ex.mean(axis=0), axis=1))

# Calculamos los logaritmos naturales de los pesos y los sumamos
log_sum_data = np.sum(np.log(data_weights))
log_sum_exp = np.sum(np.log(exp_weights))

# Calculamos la métrica Coherencia
alpha = np.exp(log_sum_data - log_sum_exp + lambertw(0.0, -np.exp(log_sum_data - log_sum_exp)).real)
alpha /= len(x)

# Imprimimos el valor de la métrica Coherencia
print("El valor de la métrica Coherencia es:", alpha)


In [ ]:
#METODO COMPLETITUD
# Creamos las instancias de datos y sus respectivas explicaciones LIME
x = np.array([[0.5, 0.7, 0.1, 0.3], [0.3, 0.9, 0.2, 0.1], [0.1, 0.2, 0.8, 0.6]])
ex = np.array([[0.2, 0.3, 0.1, 0.4], [0.3, 0.4, 0.5, 0.1], [0.1, 0.3, 0.6, 0.2]])

# Calculamos los pesos de las instancias de datos y sus respectivas explicaciones LIME
data_weights = np.exp(-np.linalg.norm(x - x.mean(axis=0), axis=1))
exp_weights = np.exp(-np.linalg.norm(ex - ex.mean(axis=0), axis=1))

# Calculamos los valores de ei y pi
ei = np.sum(exp_weights)
pi = np.sum(data_weights)

# Calculamos la métrica Completitud
gamma = ei / pi

# Imprimimos el valor de la métrica Completitud
print("El valor de la métrica Completitud es:", gamma)


In [ ]:
#METODO CONGRUENCIA
# Creamos una lista de valores de coherencia para un conjunto de N muestras
alpha_values = [0.8, 0.6, 0.7, 0.9, 0.5]

# Calculamos el valor promedio de la coherencia
alpha = np.mean(alpha_values)

# Creamos una lista de valores de alpha_i para cada instancia de datos
alpha_i_values = [0.9, 0.7, 0.6, 0.8, 0.4]

# Calculamos el número de instancias de datos
N = len(alpha_i_values)

# Calculamos la suma de las diferencias cuadráticas entre alpha_i y alpha
diff_squares_sum = np.sum((np.array(alpha_i_values) - alpha) ** 2)

# Calculamos la métrica Congruencia
delta = np.sqrt((diff_squares_sum / N) / alpha)

# Imprimimos el valor de la métrica Congruencia
print("El valor de la métrica Congruencia es:", delta)


In [ ]:
#METODO SELECTIVIDAD
# Cargamos el dataset de cáncer de mama de scikit-learn
data = load_breast_cancer()

# Definimos las características y la variable objetivo
X = data['data']
y = data['target']

# Creamos un modelo de regresión logística
model = LogisticRegression()

# Entrenamos el modelo
model.fit(X, y)

# Obtenemos las predicciones del modelo en el conjunto de datos completo
y_pred = model.predict_proba(X)[:, 1]

# Calculamos el AUC para el modelo entrenado en el conjunto de datos completo
auc_full = roc_auc_score(y, y_pred)

# Ordenamos las características de más a menos relevantes según el modelo
idx_sorted = np.argsort(np.abs(model.coef_))[0][::-1]

# Creamos una lista para almacenar los AUC para cada característica eliminada
auc_scores = []

# Iteramos sobre las características de más a menos relevantes según el modelo
for i in range(X.shape[1]):
    # Establecemos la característica i en cero
    X_test = X.copy()
    X_test[:, idx_sorted[i]] = 0
    
    # Obtenemos las predicciones del modelo en el conjunto de datos con la característica i eliminada
    y_pred_test = model.predict_proba(X_test)[:, 1]
    
    # Calculamos el AUC para el modelo entrenado en el conjunto de datos con la característica i eliminada
    auc_test = roc_auc_score(y, y_pred_test)
    
    # Almacenamos el AUC para la característica i eliminada en la lista de AUC
    auc_scores.append(auc_test)

# Calculamos la métrica Selectividad como el área bajo la curva ROC para el gráfico de AUC vs características eliminadas
selectivity = np.trapz(auc_scores, dx=1)

# Imprimimos el valor de la métrica Selectividad
print("El valor de la métrica Selectividad es:", selectivity)
